In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from sklearn.tree import DecisionTreeRegressor
import pandas as pd
import datetime

def load():
    # data_dir = "/home/asia/Documents/projects/PyCharm/IUM/data/data_v3/"
    data_dir = "/content/drive/MyDrive/Data/IUM/V3/"
    data = {}
    data["deliveries"] = pd.read_json(data_dir + "deliveries.jsonl", lines = True)
    data["sessions"] = pd.read_json(data_dir + "sessions.jsonl", lines = True)
    data["products"] = pd.read_json(data_dir + "products.jsonl", lines = True)
    data["users"] = pd.read_json(data_dir + "users.jsonl", lines = True)
    return data

def preprocess_time(deliveries: pd.DataFrame):
    deliveries["purchase_timestamp"] = pd.to_datetime ( deliveries["purchase_timestamp"], format='%Y-%m-%dT%H:%M', errors="coerce")
    deliveries['delivery_timestamp'] = pd.to_datetime ( deliveries["delivery_timestamp"], format='%Y-%m-%dT%H:%M:%S', errors="coerce")
    return deliveries


def merge(data: {}) -> pd.DataFrame:
    s = data["sessions"]
    d = data["deliveries"]
    p = data["products"]
    u = data["users"]
    purchases = s[s["purchase_id"].notnull()].copy()
    merged = pd.merge(purchases, d, on='purchase_id', how = "outer")
    merged2 = pd.merge(merged, u, on="user_id", how = "outer")
    merged_full = pd.merge(merged2, p, on="product_id", how = "left")
    return merged_full

def select_columns (merged):
    to_select = ["product_id", "offered_discount", "purchase_timestamp", "delivery_timestamp", "delivery_company", "city", "street", "product_name", "category_path", "price"]
    return merged[to_select]

In [3]:
# load data, make sure timestamps are of datetime type and merge data
data = load()
data["deliveries"] = preprocess_time(data["deliveries"])
merged = merge(data)
merged.head()

,session_id,timestamp,user_id,product_id,event_type,offered_discount,purchase_id,purchase_timestamp,delivery_timestamp,delivery_company,name,city,street,product_name,category_path,price
0,100002.0,2021-04-01 15:52:47,102,1234.0,BUY_PRODUCT,0.0,20001.0,2021-04-01 15:52:47,2021-04-02 14:06:34.432964,620.0,Aurelia Malon,Police,pl. Brzoskwiniowa 11/53,Sony DVP-SR760,Sprzęt RTV;Video;Odtwarzacze DVD,193.00
1,100003.0,2021-04-01 02:26:23,102,1289.0,BUY_PRODUCT,15.0,20002.0,2021-04-01 02:26:23,2021-04-02 10:45:27.707128,360.0,Aurelia Malon,Police,pl. Brzoskwiniowa 11/53,Philips SDV5225,Sprzęt RTV;Video;Telewizory i akcesoria;Anteny...,129.00
2,100004.0,2021-01-03 05:11:17,102,1009.0,BUY_PRODUCT,0.0,20003.0,2021-01-03 05:11:17,2021-01-04 15:43:07.635371,620.0,Aurelia Malon,Police,pl. Brzoskwiniowa 11/53,Kinect Joy Ride (Xbox 360),Gry i konsole;Gry na konsole;Gry Xbox 360,69.00
3,100005.0,2021-02-17 12:59:54,102,1001.0,BUY_PRODUCT,5.0,20004.0,2021-02-17 12:59:54,2021-02-18 11:52:42.608746,620.0,Aurelia Malon,Police,pl. Brzoskwiniowa 11/53,Telefon Siemens Gigaset DA310,Telefony i akcesoria;Telefony stacjonarne,58.97
4,100006.0,2021-02-13 09:17:52,102,1057.0,BUY_PRODUCT,5.0,20005.0,2021-02-13 09:17:52,2021-02-15 08:59:28.973167,620.0,Aurelia Malon,Police,pl. Brzoskwiniowa 11/53,Doom 3 (PC),Gry i konsole;Gry komputerowe,19.99


In [4]:
# drop that one row with user who didn't make a purchase
merged = merged.dropna()
merged.shape

(7287, 16)

In [5]:
def transform_columns(merged_df):
  merged_df["time_difference"] = merged_df["delivery_timestamp"] - merged_df["purchase_timestamp"]
  merged_df["time[min]"] = merged_df["time_difference"].apply(lambda x: int(round(x.total_seconds()/60)))
  merged_df = merged_df.drop(labels=["time_difference", "delivery_timestamp"], axis=1)

  merged_df["street_name"] = merged_df["street"].apply(lambda x: x.split(" ")[1])
  merged_df["street_name*num"] = merged_df["street"].apply(lambda x: x.split(" ")[1] + " " + x.split(" ")[2])
  merged_df = add_city_street(merged_df)
  merged_df = merged_df.drop(labels=["street"], axis=1)

  merged_df["purchase_day_of_week"] = merged_df["purchase_timestamp"].apply(lambda x: x.dayofweek)
  return merged_df

def add_city_street(merged_df):
  cities = merged_df["city"].to_list()
  streets = merged_df["street_name"].to_list()

  city_street = [cities[i] + "/" + streets[i] for i in range(len(cities))]

  merged_df["city*stret_name"] = city_street
  return merged_df

merged = select_columns(merged)
merged = transform_columns(merged)


In [6]:
merged.head()

,product_id,offered_discount,purchase_timestamp,delivery_company,city,product_name,category_path,price,time[min],street_name,street_name*num,city*stret_name,purchase_day_of_week
0,1234.0,0.0,2021-04-01 15:52:47,620.0,Police,Sony DVP-SR760,Sprzęt RTV;Video;Odtwarzacze DVD,193.00,1334,Brzoskwiniowa,Brzoskwiniowa 11/53,Police/Brzoskwiniowa,3
1,1289.0,15.0,2021-04-01 02:26:23,360.0,Police,Philips SDV5225,Sprzęt RTV;Video;Telewizory i akcesoria;Anteny...,129.00,1939,Brzoskwiniowa,Brzoskwiniowa 11/53,Police/Brzoskwiniowa,3
2,1009.0,0.0,2021-01-03 05:11:17,620.0,Police,Kinect Joy Ride (Xbox 360),Gry i konsole;Gry na konsole;Gry Xbox 360,69.00,2072,Brzoskwiniowa,Brzoskwiniowa 11/53,Police/Brzoskwiniowa,6
3,1001.0,5.0,2021-02-17 12:59:54,620.0,Police,Telefon Siemens Gigaset DA310,Telefony i akcesoria;Telefony stacjonarne,58.97,1373,Brzoskwiniowa,Brzoskwiniowa 11/53,Police/Brzoskwiniowa,2
4,1057.0,5.0,2021-02-13 09:17:52,620.0,Police,Doom 3 (PC),Gry i konsole;Gry komputerowe,19.99,2862,Brzoskwiniowa,Brzoskwiniowa 11/53,Police/Brzoskwiniowa,5


In [7]:
import numpy as np

def encode_cyclical(df):
  df["purchase_month"] = df["purchase_timestamp"].apply(lambda x: x.month)
  df["purchase_day"] = df["purchase_timestamp"].apply(lambda x: x.day)
  df["purchase_hour"] = df["purchase_timestamp"].apply(lambda x: x.hour)
  df["purchase_min"] = df["purchase_timestamp"].apply(lambda x: x.minute)

  cyclical = [("purchase_month", 12), ("purchase_day", 30), ("purchase_hour", 24), 
              ("purchase_min", 60), ("purchase_day_of_week", 7)]

  for feature, cycle in cyclical:
    df = encode_one_cyclical(df, feature, cycle)
    df = df.drop(feature, axis=1)
  df = df.drop("purchase_timestamp", axis=1)
  return df

def encode_one_cyclical(df, feature, num):
  df[feature + "_sin"] = np.sin(df[feature] * (2 * np.pi/num))
  df[feature + "_cos"] = np.cos(df[feature] * (2 * np.pi/num))
  return df

In [8]:
merged = encode_cyclical(merged)
merged.head()

,product_id,offered_discount,delivery_company,city,product_name,category_path,price,time[min],street_name,street_name*num,city*stret_name,purchase_month_sin,purchase_month_cos,purchase_day_sin,purchase_day_cos,purchase_hour_sin,purchase_hour_cos,purchase_min_sin,purchase_min_cos,purchase_day_of_week_sin,purchase_day_of_week_cos
0,1234.0,0.0,620.0,Police,Sony DVP-SR760,Sprzęt RTV;Video;Odtwarzacze DVD,193.00,1334,Brzoskwiniowa,Brzoskwiniowa 11/53,Police/Brzoskwiniowa,0.866025,-0.500000,0.207912,0.978148,-7.071068e-01,-0.707107,-0.743145,0.669131,0.433884,-0.900969
1,1289.0,15.0,360.0,Police,Philips SDV5225,Sprzęt RTV;Video;Telewizory i akcesoria;Anteny...,129.00,1939,Brzoskwiniowa,Brzoskwiniowa 11/53,Police/Brzoskwiniowa,0.866025,-0.500000,0.207912,0.978148,5.000000e-01,0.866025,0.406737,-0.913545,0.433884,-0.900969
2,1009.0,0.0,620.0,Police,Kinect Joy Ride (Xbox 360),Gry i konsole;Gry na konsole;Gry Xbox 360,69.00,2072,Brzoskwiniowa,Brzoskwiniowa 11/53,Police/Brzoskwiniowa,0.500000,0.866025,0.587785,0.809017,9.659258e-01,0.258819,0.913545,0.406737,-0.781831,0.623490
3,1001.0,5.0,620.0,Police,Telefon Siemens Gigaset DA310,Telefony i akcesoria;Telefony stacjonarne,58.97,1373,Brzoskwiniowa,Brzoskwiniowa 11/53,Police/Brzoskwiniowa,0.866025,0.500000,-0.406737,-0.913545,1.224647e-16,-1.000000,-0.104528,0.994522,0.974928,-0.222521
4,1057.0,5.0,620.0,Police,Doom 3 (PC),Gry i konsole;Gry komputerowe,19.99,2862,Brzoskwiniowa,Brzoskwiniowa 11/53,Police/Brzoskwiniowa,0.866025,0.500000,0.406737,-0.913545,7.071068e-01,-0.707107,0.978148,-0.207912,-0.974928,-0.222521


In [9]:
from sklearn.model_selection import train_test_split
categoricals = merged.select_dtypes(object).columns.to_list()
categoricals.append("delivery_company")

x_df = merged.drop("time[min]", axis=1)
y_df = merged["time[min]"]

#x_val-0.2, x_test-0.2, x_train - 0.6
x_rest, x_val, y_rest, y_val = train_test_split(x_df, y_df, test_size=0.2, random_state=78 )
x_train, x_test, y_train, y_test = train_test_split(x_rest, y_rest, test_size=0.25, random_state=78 )
print(x_test.shape, y_test.shape, x_val.shape, y_val.shape, x_train.shape, y_train.shape)

(1458, 20) (1458,) (1458, 20) (1458,) (4371, 20) (4371,)


In [10]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')

encoded_train = pd.DataFrame(encoder.fit_transform(x_train[categoricals]), index = x_train.index)
encoded_test = pd.DataFrame(encoder.transform(x_test[categoricals]), index = x_test.index)
encoded_val = pd.DataFrame(encoder.transform(x_val[categoricals]), index = x_val.index)

x_train = pd.concat([x_train.drop(categoricals, axis=1), encoded_train], axis=1)
x_test = pd.concat([x_test.drop(categoricals, axis=1), encoded_test], axis=1)
x_val = pd.concat([x_val.drop(categoricals, axis=1), encoded_val], axis=1)

print(x_test.shape, y_test.shape, x_val.shape, y_val.shape, x_train.shape, y_train.shape)

(1458, 658) (1458,) (1458, 658) (1458,) (4371, 658) (4371,)


In [11]:
x_test.dtypes

product_id            float64
offered_discount      float64
price                 float64
purchase_month_sin    float64
purchase_month_cos    float64
                       ...   
640                   float64
641                   float64
642                   float64
643                   float64
644                   float64
Length: 658, dtype: object

In [12]:
from sklearn.metrics import median_absolute_error
from sklearn.linear_model import LinearRegression

reg = LinearRegression()
reg.fit(x_train, y_train)
pred_train = reg.predict(x_train)
pred_test = reg.predict(x_test)

train_score = median_absolute_error(pred_train, y_train)
test_score = median_absolute_error(pred_test, y_test)
print(f'LinearRegression - train MAE: {train_score}, test MAE: {test_score}')


LinearRegression - train MAE: 482.32340582092456, test MAE: 537.0152715059535


In [13]:

from sklearn.tree import DecisionTreeRegressor

reg = DecisionTreeRegressor(random_state=78)
reg.fit(x_train, y_train)
pred_train = reg.predict(x_train)
pred_test = reg.predict(x_test)

train_score = median_absolute_error(pred_train, y_train)
test_score = median_absolute_error(pred_test, y_test)
print(f'Decision tree- train MAE: {train_score}, test MAE: {test_score}')


Decision tree- train MAE: 0.0, test MAE: 352.0


In [21]:
from sklearn.ensemble import RandomForestRegressor

reg = RandomForestRegressor(random_state=78)
reg.fit(x_train, y_train)
pred_train = reg.predict(x_train)
pred_test = reg.predict(x_test)

train_score = median_absolute_error(pred_train, y_train)
test_score = median_absolute_error(pred_test, y_test)
print(f'Random Forest - train MAE: {train_score}, test MAE: {test_score}')

print(reg.get_params())

Random Forest - train MAE: 117.65000000000009, test MAE: 319.8699999999999
{'bootstrap': True, 'ccp_alpha': 0.0, 'criterion': 'mse', 'max_depth': None, 'max_features': 'auto', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': 78, 'verbose': 0, 'warm_start': False}


In [27]:
params = {
    'criterion': ["mae"], 
    'max_depth': [2, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
    'max_features': ['auto', 'sqrt'],
    'min_samples_leaf': [1,2,4,8,10], 
    'min_samples_split': [2,5,10], 
    'n_estimators': [100, 150, 200, 300, 400, 500, 600, 1000]}

rs_rf = RandomizedSearchCV(reg, params, n_iter=10, cv=3, verbose=2, scoring = make_scorer(median_absolute_error))
rs_rf.fit(x_train, y_train)

best_score = rs_rf.best_score_
best_params = rs_rf.best_params_
print("Best score on cross validation: {}".format(best_score))
print("Best params: ")
for param_name in sorted(best_params.keys()):
    print('%s: %r' % (param_name, best_params[param_name]))

# #best_score = rs_clf.best_score_
# best_params = rs_clf.best_params_
# print("Best score on cross validation: {}".format(best_score))
# print("Best params: ")
# for param_name in sorted(best_params.keys()):
#     print('%s: %r' % (param_name, best_params[param_name]))

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] n_estimators=400, min_samples_split=10, min_samples_leaf=4, max_features=auto, max_depth=50, criterion=mae 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  n_estimators=400, min_samples_split=10, min_samples_leaf=4, max_features=auto, max_depth=50, criterion=mae, total=11.9min
[CV] n_estimators=400, min_samples_split=10, min_samples_leaf=4, max_features=auto, max_depth=50, criterion=mae 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 11.9min remaining:    0.0s


[CV]  n_estimators=400, min_samples_split=10, min_samples_leaf=4, max_features=auto, max_depth=50, criterion=mae, total=11.8min
[CV] n_estimators=400, min_samples_split=10, min_samples_leaf=4, max_features=auto, max_depth=50, criterion=mae 
[CV]  n_estimators=400, min_samples_split=10, min_samples_leaf=4, max_features=auto, max_depth=50, criterion=mae, total=11.8min
[CV] n_estimators=500, min_samples_split=2, min_samples_leaf=1, max_features=sqrt, max_depth=20, criterion=mae 
[CV]  n_estimators=500, min_samples_split=2, min_samples_leaf=1, max_features=sqrt, max_depth=20, criterion=mae, total= 2.0min
[CV] n_estimators=500, min_samples_split=2, min_samples_leaf=1, max_features=sqrt, max_depth=20, criterion=mae 
[CV]  n_estimators=500, min_samples_split=2, min_samples_leaf=1, max_features=sqrt, max_depth=20, criterion=mae, total= 2.0min
[CV] n_estimators=500, min_samples_split=2, min_samples_leaf=1, max_features=sqrt, max_depth=20, criterion=mae 
[CV]  n_estimators=500, min_samples_split

[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed: 137.9min finished


Best score on cross validation: 729.9133333333334
Best params: 
criterion: 'mae'
max_depth: 2
max_features: 'sqrt'
min_samples_leaf: 10
min_samples_split: 2
n_estimators: 150


In [15]:
import xgboost as xgb

xgbr = xgb.XGBRegressor(verbosity=0)
xgbr.fit(x_train, y_train)
pred_test = xgbr.predict(x_test)
pred_train = xgbr.predict(x_train)

train_score = median_absolute_error(pred_train, y_train)
test_score = median_absolute_error(pred_test, y_test)

print(f'XGBoost - train MAE: {train_score}, test MAE: {test_score}')

XGBoost - train MAE: 417.1044921875, test MAE: 460.370361328125


In [ ]:
print()